In [25]:
from functools import partial

import astroplan as ap
import astropy.units as u
import numpy as np

from astropaul.database import database_connection, html_path
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
# session.add_day_range("2025-09-29", "2025-10-02")
# session.add_day_range("2025-10-18", "2025-10-22")
# session.add_day_range("2025-11-14", "2025-11-20")
session.add_day_range("2025-12-19", "2025-12-23")
# session.add_day_range("2026-01-14", "2026-01-18")
# session.add_day_range("2026-01-27", "2026-01-31")

science_target_types = {"QuadEB", "SextEB"}
calibration_target_types = {"RV Standard", "Telluric Standard"}

phase_event_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = f"LBT Observing {session.time_range[0].iso[:10]}"

html_dir = html_path() / name
html.clear_directory(html_dir)

science_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(science_target_types)),
    tlc.add_lists,
    partial(tlc.filter_targets, criteria=lambda df: df["List LBT 2025B"]),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_database_table, table_name="Ephemerides"),
    partial(tlc.add_database_table, table_name="PEPSI Observations"),
    partial(tlc.filter_targets, criteria=lambda df: df["Num PEPSI Observations"] > 0),
    partial(tlc.add_observability, observing_session=session, calc_moon_distance=True),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 2}, **pepsi_args)),
]

calibration_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(calibration_target_types)),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_columns_from_sql, table_name="RV Calibration Targets", primary_cols=["RV"]),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.add_observability, observing_session=session, time_resolution=2 * u.hour),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 1}, **pepsi_args)),
]

common_steps = [
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    tlc.add_tess_catalog_associations,
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
]

with database_connection() as conn:
    creator = tlc.TargetListCreator(connection=conn, phase_event_defs=phase_event_defs)
    science_targets = creator.calculate(name="Science Targets", steps=science_target_steps + common_steps, verbose=False)
    calibration_targets = creator.calculate(name="Calibration Targets", steps=calibration_target_steps + common_steps, verbose=False)
    tl = tlc.TargetList.union([science_targets, calibration_targets], name=name)
    tl = creator.calculate(initial_list=tl, steps=[partial(lbt.assign_rv_standards, target_types=science_target_types)])
tl.target_list["PEPSI notes"] = tl.target_list["Target Type"]
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
tl.target_list

Name: LBT Observing 2025-12-19
Criteria:
  Union of lists: Science Targets and Calibration Targets:
    List Science Targets:
      lambda df: df["Target Type"].isin({'SextEB', 'QuadEB'})
      lambda df: df["List LBT 2025B"]
      lambda df: df["Num PEPSI Observations"] > 0
      Observability calculated at LBT in 15.0 min intervals
      AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
      Inverse of: lambda df: df["Teff"].isna()
      lambda df: (df["Observable Any Night"])
      lambda df: df["PEPSI exp_time"] < 600
    List Calibration Targets:
      lambda df: df["Target Type"].isin({'RV Standard', 'Telluric Standard'})
      Inverse of: lambda df: df["Teff"].isna()
      Observability calculated at LBT in 2.0 h intervals
      AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
      Inverse of: lambda df: df["Teff"].isna()
      lambda df: (df["Observable Any Night"])
      lambda df: df["PEPSI exp_time"] < 600
  
27 targets:
 

,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Target Source,List Featured targets,List Gemini 2024B Proposal,List Gemini 2025A Proposal,...,PEPSI cd_red,PEPSI cd_red_num_exp,PEPSI snr,PEPSI exp_time,PEPSI priority,PEPSI notes,Spectral Type,RV,RV Err,RV Standard
0,TIC 123098844,279.572833,44.698600,18:38:17.48,+44:41:54.96,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,True,False,...,6,2,100,189,(see grid),QuadEB,NaN,NaN,NaN,HIP 96306
1,TIC 260056937,63.922208,47.422197,04:15:41.33,+47:25:19.91,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,81,(see grid),QuadEB,NaN,NaN,NaN,HIP 18902
2,TIC 278352276,307.503625,48.607056,20:30:00.87,+48:36:25.4,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,True,True,...,6,2,100,94,(see grid),QuadEB,NaN,NaN,NaN,HIP 101785
3,TIC 286470992,45.330708,60.572294,03:01:19.37,+60:34:20.26,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,81,(see grid),QuadEB,NaN,NaN,NaN,HIP 10866
4,TIC 307119043,14.827542,51.221642,00:59:18.61,+51:13:17.91,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,True,False,False,...,6,2,100,61,(see grid),QuadEB,NaN,NaN,NaN,HIP 1127
5,TIC 317863971,110.567500,3.031925,07:22:16.2,+03:01:54.93,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,81,(see grid),QuadEB,NaN,NaN,NaN,HIP 39468
6,TIC 322727163,309.716625,50.466819,20:38:51.99,+50:28:00.55,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,154,(see grid),QuadEB,NaN,NaN,NaN,HIP 101785
7,TIC 367448265,78.382458,35.653053,05:13:31.79,+35:39:10.99,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,60,(see grid),QuadEB,NaN,NaN,NaN,HIP 26981
8,TIC 389836747,23.295458,61.585308,01:33:10.91,+61:35:07.11,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,116,(see grid),QuadEB,NaN,NaN,NaN,HIP 10866
9,TIC 392229331,54.767917,61.064203,03:39:04.3,+61:03:51.13,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,False,False,False,...,6,2,100,108,(see grid),QuadEB,NaN,NaN,NaN,HIP 18902


In [27]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.85),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=30 * u.min)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_event_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
pl.categorize_priorities(
    bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
)

 '2025-12-19T02:00:00.000000000' '2025-12-19T02:30:00.000000000'
 '2025-12-19T03:00:00.000000000' '2025-12-19T03:30:00.000000000'
 '2025-12-19T04:00:00.000000000' '2025-12-19T04:30:00.000000000'
 '2025-12-19T05:00:00.000000000' '2025-12-19T05:30:00.000000000'
 '2025-12-19T06:00:00.000000000' '2025-12-19T06:30:00.000000000'
 '2025-12-19T07:00:00.000000000' '2025-12-19T07:30:00.000000000'
 '2025-12-19T08:00:00.000000000' '2025-12-19T08:30:00.000000000'
 '2025-12-19T09:00:00.000000000' '2025-12-19T09:30:00.000000000'
 '2025-12-19T10:00:00.000000000' '2025-12-19T10:30:00.000000000'
 '2025-12-19T11:00:00.000000000' '2025-12-19T11:30:00.000000000'
 '2025-12-19T12:00:00.000000000' '2025-12-19T12:30:00.000000000'
 '2025-12-19T13:00:00.000000000' '2025-12-19T13:30:00.000000000'], obsgeoloc=[( 5339770.6978711 , -684316.86393081, 3414245.14326175),
 ( 5383491.04897879,   19314.07380387, 3414112.13443666),
 ( 5334743.3419456 ,  722614.59791955, 3414213.26118185),
 ( 5194366.23106227, 1413485.15394

In [28]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)
with open("UVa_Multistar.README", "w") as f:
    f.write(readme)

other_files = {
    "LBT Readme": lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session),
    "LBT CSV": open("UVa_Multistar.csv", "r").readlines(),
}

html.render_observing_pages(tl, pl, other_files, html_dir)
# pl.categorical_priorities[0]

In [29]:
# import asyncio
# import glob
# # for categorical_file in glob.glob("*.*"):

# for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-07-04/Categorical Priorities *.html"):

#     await html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
#     print(categorical_file)